# <font color=red> Step 1. </font> Training Data

In [10]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

# 為資源的 Azure 端點和訂用帳戶金鑰建立變數 =========================================================
ENDPOINT = "yours"
training_key = "yours"

#建立新的自訂視覺專案 ==============================================================================

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project_name = 'tainanfood'
project = trainer.create_project(project_name)


#將標記新增至專案==================================================================================
# Make two tags in the new project
sandwich_tag = trainer.create_tag(project.id, "sandwich")
beef_tag = trainer.create_tag(project.id, "beef")
ice_tag = trainer.create_tag(project.id, "ice")
corn_tag = trainer.create_tag(project.id, "corn")
wagre_tag = trainer.create_tag(project.id, "wagre")

#上傳和標記影像====================================================================================
# You can get the images for this sample at:
# https://github.com/Azure-Samples/cognitive-services-sample-data-files/tree/master/CustomVision/ImageClassification/Images
base_image_location = 'C:/Desktop/trainingdata/'

print("Adding images...")

image_list = []

for image_num in range(1, 6):
    file_name = "sandwich({}).jpg".format(image_num)
    with open(os.path.join (base_image_location, "sandwich/", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[sandwich_tag.id]))

for image_num in range(1, 6):
    file_name = "beef({}).jpg".format(image_num)
    with open(os.path.join (base_image_location, "beef/", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[beef_tag.id]))
for image_num in range(1, 6):
    file_name = "ice({}).jpg".format(image_num)
    with open(os.path.join (base_image_location, "ice/", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[ice_tag.id]))
for image_num in range(1, 6):
    file_name = "corn({}).jpg".format(image_num)
    with open(os.path.join (base_image_location, "corn/", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[corn_tag.id]))
for image_num in range(1, 6):
    file_name = "wagre({}).jpg".format(image_num)
    with open(os.path.join (base_image_location, "wagre/", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[wagre_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)
    

#為專案定型======================================================================================
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
#     print ("Training status: " + iteration.status)
#     print ("Waiting 10 seconds...")
    time.sleep(10)
print('End Training ！')    

Creating project...
Adding images...
Training...
End Training ！


# <font color=red> Step 2. </font> Database
## 讀取整理過後的excel

In [14]:
import pandas as pd

values = tuple()
valuesList = list()

df = pd.read_excel("./store.xlsx",engine='openpyxl')

for n in range(0,41):
    values =  df.index[n] , df['店名'][n] , df['地址'][n] ,df['均消'][n] ,df['店評價'][n] ,df['tag'][n] 
    valuesList.append(values)
    values = tuple()
valuesList

[(0, '品馨冰菓室', '臺南市中西區中山路128號', '無', 5.0, '冰品'),
 (1, '島入冰店', '臺南市中西區赤崁東街15號巷2號', '均消 $120', 4.9, '冰品'),
 (2, '牛狀元牛肉湯', '臺南市麻豆區麻佳路一段87之7號', '均消 $200', 4.5, '牛肉湯'),
 (3, '美雪冰菓室', '臺南市東區裕誠街312號', '均消 $60', 4.9, '冰品'),
 (4, '森茂碗粿', '臺南市中西區開山路317號', '均消 $30', 4.0, '碗粿'),
 (5, '牛口水牛肉湯', '臺南市東區中華東路二段73巷1號', '無', 4.3, '牛肉湯'),
 (6, '清水堂愛玉專賣店', '臺南市中西區中正路305號', '均消 $100', 4.6, '冰品'),
 (7, '南都石頭鄉玉米', '臺南市中西區康樂街127號', '均消 $80', 4.6, '玉米'),
 (8, '鬍鬚忠牛肉湯', '臺南市中西區民族路三段91號', '均消 $150', 4.5, '牛肉湯'),
 (9, '阿嬤碳烤三明治', '臺南市中西區永福路二段227巷3號', '均消 $35', 4.1, '三明治'),
 (10, '一味品碗粿魚羹', '臺南市中西區國華街三段177號', '均消 $50', 4.6, '碗粿'),
 (11, '周家古早味碗粿/肉粽', '臺南市東區榮譽街21號', '均消 $80', 3.7, '碗粿'),
 (12, '上舜碗粿', '臺南市中西區開山路9號', '均消 $30', 4.4, '碗粿'),
 (13, '二牛牛肉湯', '臺南市安平區安平路570號', '均消 $350', 4.2, '牛肉湯'),
 (14, '文章牛肉湯', '臺南市安平區安平路590號', '均消 $150', 4.3, '牛肉湯'),
 (15, '富盛號碗粿', '臺南市中西區民族路3段11號', '均消 $35', 4.3, '碗粿'),
 (16, '阿順冰菓室', '臺南市北區開元路364號', '均消 $200', 4.5, '冰品'),
 (17, '盛牛肉湯', '臺南市中西區國華街二段81巷4號', '均消 $100', 4.0, '牛肉湯'),
 (18, 

## 存成 MySQL
#### 1. 先在workbench執行Database.sql，建立資料庫及表格
#### 2. 第8行填寫workbench密碼

In [23]:
import pymysql


# 設定資料庫連線資訊
host = 'localhost'
port = 3306
user = 'root'
passwd = 'password'
db = 'azure'
charset = 'utf8mb4'

# 建立連線,要先將database建立好
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)

# 建立游標
cursor = conn.cursor()

# 先寫好 SQL 語法
sql = """
INSERT INTO store (ID, Name, Address, Price, Score, Tag)
VALUES (%s, %s, %s, %s, %s, %s);
"""

# 將指令放進 cursor 物件，並執行
cursor.executemany(sql,valuesList)

# 因 pymysql 預設不會自動 commit，故要記得 commit
conn.commit()
# 關閉連線
cursor.close()
conn.close()

# <font color=red> Step 3. </font> Data presentation
### 使用templates裡的html做輸出

In [46]:
from flask import Flask,render_template
from flask_uploads import UploadSet, IMAGES, configure_uploads
from flask import request
from msrest.authentication import ApiKeyCredentials
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import os

ENDPOINT = "yours"
prediction_resource_id = "yours"
publish_iteration_name = "classifyModel"
prediction_key = "yours"
new_image_location = 'C:/Desktop/static/images/'

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)



app = Flask(__name__)

app.config['SECRET_KEY'] = 'development'
app.config['UPLOADED_DEF_DEST'] = r'C:/Desktop/static/images/'
app.config['UPLOAD_FOLDER'] = os.getcwd()+'\static\images'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

abc = UploadSet(name='def', extensions=IMAGES)
configure_uploads(app, abc)

html = """
<!DOCTYPE html>
<title>台南美食推推</title>
<html lang="en">
    <h1>台南美食推推</h1>
    <form action="/uploads/" enctype=multipart/form-data method="POST">
        <input type=file name=in_abc>
        <input type=submit value='上傳'>
    </form>
</html>
"""

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],filename)

@app.route('/uploads/', methods=['GET', 'POST'])
def uploads():
   
    if request.method == 'POST' and 'in_abc' in request.files:
        file = request.files['in_abc']
        
        if file and allowed_file(file.filename):
            filename = abc.save(file)
    #         print(filename)
    #         file_url = abc.url(filename)
    #         print(file_url)
            file_url = url_for('uploaded_file', filename=filename)
            imghtml = html + '<br><img src=' + file_url + '>'

            with open(os.path.join (new_image_location, filename), "rb") as image_contents:
                results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())
            foodname = results.predictions[0].tag_name

            if foodname == 'beef': root = imghtml + render_template('beef.html') 
            if foodname == 'sandwich': root = imghtml + render_template('sandwich.html') 
            if foodname == 'ice': root = imghtml + render_template('ice.html') 
            if foodname == 'corn': root = imghtml + render_template('corn.html') 
            if foodname == 'wagre': root = imghtml + render_template('wagre.html')            
        else:
            root = html + """
            <h1><font color=red>請上傳圖片類型檔案!</font></h1>
            <h3>例如：.png 、 .jpg 、 .jpeg 、 .gif</h3>
            """
    else:
        root = html
        
    return root

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2021 21:48:24] "GET /uploads/ HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2021 21:48:27] "POST /uploads/ HTTP/1.1" 200 -


### 使用MySQL裡的資料做輸出

In [1]:
from flask import Flask,render_template
from flask_uploads import UploadSet, IMAGES, configure_uploads
from flask import request
from msrest.authentication import ApiKeyCredentials
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import os
import pymysql

ENDPOINT = "yours"
prediction_resource_id = "yours"
publish_iteration_name = "classifyModel"
prediction_key = "yours"
new_image_location = 'C:/Desktop/static/images/'

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)



app = Flask(__name__)

app.config['SECRET_KEY'] = 'development'
app.config['UPLOADED_DEF_DEST'] = r'C:/Desktop/static/images/'
app.config['UPLOAD_FOLDER'] = os.getcwd()+'\static\images'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

abc = UploadSet(name='def', extensions=IMAGES)
configure_uploads(app, abc)

html = """
<!DOCTYPE html>
<title>台南美食推推</title>
<html lang="en">
    <h1>台南美食推推</h1>
    <form action="/uploads/" enctype=multipart/form-data method="POST">
        <input type=file name=in_abc>
        <input type=submit value='上傳'>
    </form>
</html>
"""

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],filename)

@app.route('/uploads/', methods=['GET', 'POST'])
def uploads():
   
    if request.method == 'POST' and 'in_abc' in request.files:
        file = request.files['in_abc']
        
        if file and allowed_file(file.filename):
            filename = abc.save(file)
    #         print(filename)
    #         file_url = abc.url(filename)
    #         print(file_url)
            file_url = url_for('uploaded_file', filename=filename)
            imghtml = html + '<br><img src=' + file_url + '><br>'

            with open(os.path.join (new_image_location, filename), "rb") as image_contents:
                results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())
            foodname = results.predictions[0].tag_name

# MySQL===================================================================
            conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='password', db='azure', charset='utf8mb4')
            cursor = conn.cursor()

            if foodname == 'beef': chfoodname = '牛肉湯'
            if foodname == 'sandwich': chfoodname = '三明治'
            if foodname == 'ice': chfoodname = '冰品'
            if foodname == 'corn': chfoodname = '玉米'
            if foodname == 'wagre': chfoodname = '碗粿' 

            sql = """select Name, Address, Price 
            from store 
            where Tag = '%s' 
            order by Score desc 
            limit 5;
            """%(chfoodname)

            cursor.execute(sql)

            data = cursor.fetchall()

            name = """
            <h3>上傳檔案為：%s</h3>
            <h1><font color=red>%s推薦 !</font></h1>    
            """%(chfoodname,chfoodname)
            imghtml = imghtml + name
            
            number = 1

            for outdata in range(0,5):
                storename = """
                <br><b>%s.%s</b>
                <br>地址：%s<br>        
                """%(number,data[outdata][0],data[outdata][1])
                imghtml = imghtml + storename
                storename = ''
                number += 1
            
            root = imghtml
            
            conn.commit()
            cursor.close()
            conn.close()     
# MySQL===================================================================
        else:
            root = html + """
            <h1><font color=red>請上傳圖片類型檔案!</font></h1>
            <h3>例如：.png 、 .jpg 、 .jpeg 、 .gif</h3>
            """
    else:
        root = html
        
    return root

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2021 22:19:12] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Apr/2021 22:19:20] "GET /uploads/ HTTP/1.1" 200 -
[2021-04-24 22:19:24,668] ERROR in app: Exception on /uploads/ [POST]
Traceback (most recent call last):
  File "c:\program files\python36\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\program files\python36\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\program files\python36\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\program files\python36\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\program files\python36\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\program files\python36\lib\site-package